## Env Related

### Reduce DataFrame Memory Usage

In [1]:
def reduce_mem_usage(df, use_float16=False):
    from pandas.api.types import is_datetime64_any_dtype as is_datetime
    from pandas.api.types import is_categorical_dtype
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Parallelize DataFrame

In [2]:
def df_parallelize_run(df, func):
    import multiprocessing
    num_partitions, num_cores = psutil.cpu_count(), psutil.cpu_count()
    df_split = np.array_split(df, num_partitions)
    pool = multiprocessing.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

## Feature Related (Categorical)

### Concatenate Categorical Columns

In [3]:
def category_concat(df, subject_cols, print_option=True):
    na_col = list(df.columns[df.isna().any()])
    for col in na_col:
        df[col].fillna('', inplace=True)
    temp_str = ''
    for col in subject_cols:
        temp_str += '_' + col
    df[temp_str[1:]] = ''
    for col in subject_cols:
        df[temp_str[1:]] += df[col]
    
    if print_option:
        print("Generated features: category_concat")
        print(f"'{temp_str[1:]}',")
        print()
    del na_col, temp_str, col; gc.collect()

### Target Encode (with smooth / without OOF consideration)

In [4]:
def apply_target_encode(df, test_df, target_col, cat_col, smooth=False, m=0, statistic=False, print_option=True):
    # df[target_col] = np.log1p(df[target_col])
    df_group = df.groupby(cat_col)[target_col]
    group_mean = df_group.mean().astype(np.float16)
    
    # ===== smoothing =====
    if smooth:
        global_mean = df[target_col].mean()
        group_count = df_group.count().astype(np.float16)
        smoother = ((group_count * group_mean) + (m * global_mean)) / (group_count + m)
        df[f'{cat_col}_smoothed_mean'] = df[f'{cat_col}'].map(smoother)
        test_df[f'{cat_col}_smoothed_mean'] = test_df[f'{cat_col}'].map(smoother)
        del global_mean, group_count, smoother; gc.collect()

    # ===== no smoothing =====
    elif smooth == False:
        df[f'{cat_col}_mean'] = df[f'{cat_col}'].map(group_mean)
        test_df[f'{cat_col}_mean'] = test_df[f'{cat_col}'].map(group_mean)
        
    # ===== more target statistic =====
    if statistic:
        group_min = df_group.min().astype(np.float16)
        group_max = df_group.max().astype(np.float16)
        group_std = df_group.std().astype(np.float16)
        
        df[f'{cat_col}_min'] = df[f'{cat_col}'].map(group_min)
        test_df[f'{cat_col}_min'] = df[f'{cat_col}'].map(group_min)
        
        df[f'{cat_col}_max'] = df[f'{cat_col}'].map(group_max)
        test_df[f'{cat_col}_max'] = df[f'{cat_col}'].map(group_max)
        
        df[f'{cat_col}_std'] = df[f'{cat_col}'].map(group_std)
        test_df[f'{cat_col}_std'] = df[f'{cat_col}'].map(group_std)
        
        df[f'{cat_col}_range'] = df[f'{cat_col}_max'] - df[f'{cat_col}_min']
        test_df[f'{cat_col}_range'] = df[f'{cat_col}_max'] - df[f'{cat_col}_min']
        
        group_Q1 = df_group.quantile(0.25).astype(np.float16)
        group_Q2 = df_group.median().astype(np.float16)
        group_Q3 = df_group.quantile(0.75).astype(np.float16)
        
        df[f'{cat_col}_Q1'] = df[f'{cat_col}'].map(group_Q1)
        test_df[f'{cat_col}_Q1'] = test_df[f'{cat_col}'].map(group_Q1)
        
        df[f'{cat_col}_Q2'] = df[f'{cat_col}'].map(group_Q2)
        train_df[f'{cat_col}_Q2'] = train_df[f'{cat_col}'].map(group_Q2)
        
        df[f'{cat_col}_Q3'] = df[f'{cat_col}'].map(group_Q3)
        test_df[f'{cat_col}_Q3'] = test_df[f'{cat_col}'].map(group_Q3)
        
        df[f'{cat_col}_IQR'] = df[f'{cat_col}_Q1'] - df[f'{cat_col}_Q3']
        test_df[f'{cat_col}_IQR'] = test_df[f'{cat_col}_Q1'] - test_df[f'{cat_col}_Q3']
        
        del group_min, group_max, group_std, group_Q1, group_Q2, group_Q3; gc.collect()
        
    if print_option:
        print("Generated features: apply_target_encode")
        if smooth:
            print(f"'{cat_col}_smoothed_mean',")
            print()
        elif smooth == False:
            print(f"'{cat_col}_mean',")
            print()
        if statistic:
            print(f"'{cat_col}_min',")
            print(f"'{cat_col}_max',")
            print(f"'{cat_col}_std',")
            print(f"'{cat_col}_range',")
            print(f"'{cat_col}_Q1',")
            print(f"'{cat_col}_Q2',")
            print(f"'{cat_col}_Q3',")
            print(f"'{cat_col}_IQR',")
            print()
    del df_group, group_mean; gc.collect()

### Label Encode (not ordinal)

In [5]:
def apply_label_encode(df, subject_cols):
    for str_col in subject_cols:
        # ===== assumes Series of string =====
        temp_dict = {value: i for i, value in enumerate(df[str_col].unique())}
        df[str_col] = df[str_col].map(temp_dict)
    del temp_dict, str_col; gc.collect()

### Ratio Encode

In [6]:
def apply_freq_encode(df, str_col, print_option=True):
    temp_dict = {sample: df.loc[df[str_col]==sample].shape[0] for sample in df[str_col].unique()}
    df[f'{str_col}_ratio'] = df[str_col].map(temp_dict) / df[str_col].shape[0]
    
    if print_option:
        print("Generated features: apply_freq_encode")
        print(f"'{str_col}_ratio'")
        print()
    del temp_dict; gc.collect()

## Feature Related (Numerical)

In [7]:
# aggs = {
#     'uid': ['count'],
#     'is_manual': ['sum', 'mean'],
#     'elapsed_days_succeeded_created': ['mean', 'std', 'max', 'min'],
#     'elapsed_days_created_premium': ['mean', 'std', 'max', 'min'],
#     'elapsed_days_created': ['mean', 'std', 'max', 'min'],
#     'elapsed_days_succeeded_premium': ['mean', 'std', 'max', 'min'],
#     'elapsed_days_succeeded': ['mean', 'std', 'max', 'min'],
#     'created_before_premium': ['sum', 'mean'],
#     'created_after_premium': ['sum', 'mean'],
#     'succeeded_before_premium': ['sum', 'mean'],
#     'succeeded_before_premium': ['sum', 'mean'],
# }
# aggs.update({col: ['sum', 'mean'] for col in service_category_id_cols})

# group_account_df = account_df.groupby(ID).agg(aggs)
# group_account_df.columns = [f'{k}_{v.upper()}' for k, vs in aggs.items() for v in vs]
# group_account_df = group_account_df.reset_index()

### Cyclical Encode

In [8]:
def apply_cyclical(df, str_col):
    # df["hour"] = df["timestamp"].dt.hour
    # ===== assumes integer array =====
    df -= df[str_col].min()
    int_max = df[str_col].max()
    df[f'{str_col}_sin'] = np.sin(2 * np.pi * df[str_col] / int_max)
    df[f'{str_col}_cos'] = np.cos(2 * np.pi * df[str_col] / int_max)
    del int_max; gc.collect()

### Rolling Statistic

In [9]:
def apply_mov_stat(df, str_col, list_windows, fix=False, print_option=True):

    # ===== assumes timestamp is aligned =====
    for win in list_windows:
        rolled = df[str_col].rolling(window=win, min_periods=0)
        mov_avg = rolled.mean().reset_index() #.astype(np.float16)
        mov_max = rolled.max().reset_index() #.astype(np.float16)
        mov_min = rolled.min().reset_index() #.astype(np.float16)
        mov_std = rolled.std().reset_index() #.astype(np.float16)
        # mov_Q1 = rolled.quantile(0.25).reset_index() #.astype(np.float16)
        # mov_Q2 = rolled.quantile(0.5).reset_index() #.astype(np.float16)
        # mov_Q3 = rolled.quantile(0.75).reset_index() #.astype(np.float16)

        if fix:
            formula = int((win/2) - win)
            df[f'{str_col}_movavg_{win}'] = mov_avg[f'{str_col}'].shift(formula)
            df[f'{str_col}_movmax_{win}'] = mov_max[f'{str_col}'].shift(formula)
            df[f'{str_col}_movmin_{win}'] = mov_min[f'{str_col}'].shift(formula)
            df[f'{str_col}_movstd_{win}'] = mov_std[f'{str_col}'].shift(formula)
            # df[f'{str_col}_movQ1_{win}'] = mov_Q1[f'{str_col}'].shift(formula)
            # df[f'{str_col}_movQ2_{win}'] = mov_Q2[f'{str_col}'].shift(formula)
            # df[f'{str_col}_movQ3_{win}'] = mov_Q3[f'{str_col}'].shift(formula)
            print()
            del formula
        else:
            df[f'{str_col}_movavg_{win}'] = mov_avg[f'{str_col}']
            df[f'{str_col}_movmax_{win}'] = mov_max[f'{str_col}']
            df[f'{str_col}_movmin_{win}'] = mov_min[f'{str_col}']
            df[f'{str_col}_movstd_{win}'] = mov_std[f'{str_col}']
            # df[f'{str_col}_movQ1_{win}'] = mov_Q1[f'{str_col}']
            # df[f'{str_col}_movQ2_{win}'] = mov_Q2[f'{str_col}']
            # df[f'{str_col}_movQ3_{win}'] = mov_Q3[f'{str_col}']
            print()
        
        if print_option:
            print('Generated features: apply_mov_stat')
            print(f"'{str_col}_movavg_{win}',")
            print(f"'{str_col}_movmax_{win}',")
            print(f"'{str_col}_movmin_{win}',")
            print(f"'{str_col}_movstd_{win}',")
            #print(f"'{str_col}_movQ1_{win}',")
            #print(f"'{str_col}_movQ2_{win}',")
            #print(f"'{str_col}_movQ3_{win}',")
            print()
            
    del win, rolled, mov_avg, mov_max, mov_min, mov_std; gc.collect()
    # del mov_Q1, mov_Q2, mov_Q3; gc.collect()

### Nonlinear (log1p) Transformation

In [10]:
def apply_nonlinear(df, subject_cols):
    for col in subject_cols:
        temp_count = df[f'{col}'].isna().sum()
        df[f'{col}'] = np.log1p(df[f'{col}'])
        if df[f'{col}'].isna().sum() > temp_count:
            print(f"New nan in '{col}' via apply_nonlinear")
    del col, temp_count; gc.collect()

### Shift Features

In [11]:
def apply_shift_feature(df, subject_cols, list_shift, print_option=True):
    for col in subject_cols:
        for step in list_shift:
            df[f'{col}_shift_{step}'] = df[col].shift(int(step))
            
    if print_option:
        print('Generated features: apply_shift_feature')
        for col in subject_cols:
            for step in list_shift:
                print(f"'{col}_shift_{step}',")
        print()
        
    del col, step; gc.collect()

### Oneth Feature

In [12]:
def apply_oneth_feature(df, str_col, print_option=True):
    import math
    modify = np.vectorize(math.modf)
    oneth, tenth = modify(df[str_col] / 10)
    df[f'{str_col}_oneth'] = oneth * 10
    
    if print_option:
        print('Generated features: apply_oneth_feature')
        print(f"'{str_col}_oneth',")
        print()
        
    del tenth; gc.collect()

### nan Binary Features

In [13]:
def apply_isna_feature(df, subject_cols, print_option=True):
    binary_isna = [col+"_isnan" for col in subject_cols]
    df[binary_isna] = df[subject_cols].isna().astype(int)
    
    if print_option:
        print('Generated features: apply_oneth_feature')
        for col in binary_isna:
            print(f"'{col}',")
        print()
        
    del binary_isna; gc.collect()

### Row nan Count Feature

In [14]:
def apply_row_nan(df, print_option=True):
    df['row_nan'] = df.isna().sum(axis=1).astype(np.int8)
    
    if print_option:
        print('Generated features: apply_row_nan')
        print("'row_nan',")
        print()

### Bruteforce Combination

In [15]:
def bruteforce_combination(df, subject_cols, choose=2, print_option=True):
    from itertools import combinations
    comb = combinations(subject_cols, choose)
    for feat_1, feat_2 in comb:
        df[f'{feat_1}_.+_{feat_2}'] = df[f'{feat_1}'] + df[f'{feat_1}']
        df[f'{feat_1}_.-_{feat_2}'] = df[f'{feat_1}'] - df[f'{feat_1}']
        df[f'{feat_1}_.*_{feat_2}'] = df[f'{feat_1}'] * df[f'{feat_1}']
        df[f'{feat_1}_./_{feat_2}'] = df[f'{feat_1}'] / df[f'{feat_1}']
            
    if print_option:
        print('Generated features: bruteforce_feature_combination')
        for feat_1, feat_2 in comb:
            print(f"'{feat_1}_.+_{feat_2}',")
            print(f"'{feat_1}_.-_{feat_2}',")
            print(f"'{feat_1}_.*_{feat_2}',")
            print(f"'{feat_1}_./_{feat_2}',")
        print()
            
    del comb, feat_1, feat_2; gc.collect()

## Data Related

### Clipping

In [16]:
def apply_clip(df, str_col, pct_lower, pct_upper):
    LB, UB = np.percentile(df[str_col], [pct_lower, pct_upper])
    df[str_col] = np.clip(df[str_col], LB, UB)
    del LB, UB; gc.collect()

### Interpolation

In [17]:
def apply_interpolation(df, subject_cols, int_order, supp_median_fill=False):
    lin = lambda var: var.interpolate(method='linear', limit_direction='both')
    pol = lambda var: var.interpolate(method='polynomial', order=int_order, limit_direction='both')
    
    # ===== in ASHRAE, grouping was done via site_id =====
    # linear = df.groupby(grouping_col).apply(lin)
    # polyno = df.groupby(grouping_col).apply(pol)
    
    linear = df[subject_cols].apply(lin)
    polyno = df[subject_cols].apply(pol)
    df[subject_cols] = (linear[subject_cols] + polyno[subject_cols]) * 0.5
    
    # ===== if missing value remains: =====
    if supp_median_fill:
        #[col for col in cols if temp[col].isna().sum() > 0]
        for col in subject_cols:
            df[col].fillna(df[col].median(), inplace=True)
            del col
    del lin, pol, linear, polyno; gc.collect()

### Adversarial Validation with LightGBM

In [18]:
def advarsarial_validation_lightgbm(
    params,
    X_train,
    X_test,
    features,
    categorical=[],
    n_splits=5,
    shuffle=True,
    seed=42,
):
    X_train_adv = X_train.copy()
    X_test_adv = X_test.copy()
    
    X_train_adv['test'] = 0
    X_test_adv['test'] = 1
    
    X_train_adv = pd.concat([X_train_adv, X_test_adv], axis=0).reset_index(drop=True)
    y_train_adv = X_train_adv['test']
    X_train_adv = X_train_adv.drop('test', axis=1)
    
    printl(f'{X_train_adv.shape}, {y_train_adv.shape}, {len(features)}')
    
    cv = build_cv_spliter(X_train_adv,
                          y_train_adv,
                          strategy='stratified',
                          n_splits=n_splits,
                          shuffle=shuffle,
                          random_seed=seed)
    
    adv_metrics = {'AUC': roc_auc_score}
    _, adv, feature_importance_df = run_kfold_lightgbm(params,
                                                       X_train_adv,
                                                       y_train_adv,
                                                       X_train,
                                                       cv,
                                                       features,
                                                       adv_metrics,
                                                       categorical=cat_features)
    
    return adv, feature_importance_df

## Modelling Related

### LightGBM (Binary Classification: max auc)

In [19]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from bayes_opt import BayesianOptimization

In [20]:
def lgb_train_auc(train_set, valid_set, metric,
                  lr, nl, md, bf, ff, mcw, mdil, l1, l2):
    params = {'objective': 'binary',
              'metric': metric,
              'boosting': 'gbdt',
              'seed': 8982,
              'learning_rate': lr,
              'num_leaves': int(nl),
              'max_depth': int(md),
              'bagging_freq': int(5),
              'bagging_fraction': bf,
              'feature_fraction': ff,
              'min_child_weight': mcw,   
              'min_data_in_leaf': int(mdil),
              'lambda_l1': l1,
              'lambda_l2': l2}

    # ===== train_set: expects tuple (X_train, y_train) =====
    X_train, y_train = train_set
    # ===== valid_set: expects tuple (X_valid, y_valid) =====
    X_valid, y_valid = valid_set

    # ===== define data in lgb terms =====
    d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=category_cols)
    d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=category_cols)
    watchlist = [d_train, d_valid]

    learning_curve = {}
    model = lgb.train(params,
                      train_set=d_train,
                      valid_sets=watchlist,
                      num_boost_round=600,
                      evals_result=learning_curve,
                      verbose_eval=200,
                      early_stopping_rounds=20)
    
    best_score = {f'train_{metric}': model.best_score['training'][f'{metric}'],
                  f'valid_{metric}': model.best_score['valid_1'][f'{metric}']}
    return model, best_score, learning_curve

def lgb_skfold(X_train, y_train,
              learning_rate=0.05, num_leaves=31, max_depth=-1,
              bagging_fraction=0.9, feature_fraction=0.9,
              min_child_weight=1e-3, min_data_in_leaf=20,
              lambda_l1=0.0, lambda_l2=0.0,
              bayes_opt=True):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=8982)
    metric = 'auc'
    #cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
    #print(cat_features)
    
    oofs = np.zeros(X_train.shape[0],)
    models = []; learning_curves = []; valid_score = []; # best_scores = []
    for train_idx, valid_idx in kf.split(X_train, y_train):
        train_data = X_train.iloc[train_idx,:], y_train[train_idx]
        valid_data = X_train.iloc[valid_idx,:], y_train[valid_idx]

        model, best_score, learning_curve = lgb_train_auc(train_set=train_data,
                                                          valid_set=valid_data,
                                                          metric=metric,
                                                          lr=learning_rate,
                                                          nl=num_leaves,
                                                          md=max_depth,
                                                          bf=bagging_fraction,
                                                          ff=feature_fraction,
                                                          mcw=min_child_weight,
                                                          mdil=min_data_in_leaf,
                                                          l1=lambda_l1,
                                                          l2=lambda_l2)
        oofs[valid_idx] = model.predict(X_train.iloc[valid_idx,:], num_iteration=model.best_iteration)
        models.append(model)
        learning_curves.append(learning_curve)
        valid_score.append(best_score[f'valid_{metric}'])
        # best_scores.append(best_score)
        gc.collect()
    
    valid_score_avg = np.mean(valid_score)
    valid_score_std = np.std(valid_score)
    
    if bayes_opt:
        return valid_avg_score
    else:
        return valid_score_avg, valid_score_std, models, oofs #best_scores, learning_curves

def lgb_clf_bayes_opt(init_points=20, n_iteration=80):
    bounds = {'learning_rate': (0.001, 0.3),
              'num_leaves': (20, 500), 
              'bagging_fraction' : (0.1, 1),
              'feature_fraction' : (0.1, 1),
              'min_child_weight': (0.001, 0.99),   
              'min_data_in_leaf': (3, 200),
              'max_depth': (-1, 100),
              'lambda_l1': (0.1, 300), 
              'lambda_l2': (0.1, 300)}
    
    optimizer = BayesianOptimization(f=lgb_skfold, pbounds=bounds, random_state=8982)
    optimizer.maximize(init_points=init_points, n_iter=n_iteration)
    
    print('Best score:', optimizer.max['target'])
    print('Best set of parameters:')
    print(optimizer.max['params'])
    param = optimizer.max['params']; cv = optimizer.max['target']
    return param, cv

### LightGBM (Regression: min rmse)

In [21]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from bayes_opt import BayesianOptimization

In [22]:
def lgb_train(train_set, valid_set, metric,
              lr, nl, md, bf, ff, mcw, mdil, l1, l2):
    params = {'objective': 'regression',
              'metric': metric,
              'boosting': 'gbdt',
              'seed': 8982,
              'learning_rate': lr,
              'num_leaves': int(nl),
              'max_depth': int(md),
              'bagging_freq': int(5),
              'bagging_fraction': bf,
              'feature_fraction': ff,
              'min_child_weight': mcw,   
              'min_data_in_leaf': int(mdil),
              'lambda_l1': l1,
              'lambda_l2': l2}

    # ===== train_set: expects tuple (X_train, y_train) =====
    X_train, y_train = train_set
    # ===== valid_set: expects tuple (X_valid, y_valid) =====
    X_valid, y_valid = valid_set

    # ===== define data in lgb terms =====
    d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=category_cols)
    d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=category_cols)
    watchlist = [d_train, d_valid]

    learning_curve = {}
    model = lgb.train(params,
                      train_set=d_train,
                      valid_sets=watchlist,
                      num_boost_round=600,
                      evals_result=learning_curve,
                      verbose_eval=200,
                      early_stopping_rounds=20)
    
    best_score = {f'train_{metric}': model.best_score['training'][f'{metric}'],
                  f'valid_{metric}': model.best_score['valid_1'][f'{metric}']}
    return model, best_score, learning_curve

def lgb_kfold(X_train, y_train,
              learning_rate=0.05, num_leaves=31, max_depth=-1,
              bagging_fraction=0.9, feature_fraction=0.9,
              min_child_weight=1e-3, min_data_in_leaf=20,
              lambda_l1=0.0, lambda_l2=0.0,
              bayes_opt=True):
    kf = KFold(n_splits=5, shuffle=True, random_state=8982)
    metric = 'rmse'
    #cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
    #print(cat_features)
    
    oofs = np.zeros(X_train.shape[0],)
    models = []; learning_curves = []; valid_score = []; # best_scores = []
    for train_idx, valid_idx in kf.split(X_train, y_train):
        train_data = X_train.iloc[train_idx,:], y_train[train_idx]
        valid_data = X_train.iloc[valid_idx,:], y_train[valid_idx]

        model, best_score, learning_curve = lgb_train(train_set=train_data,
                                                      valid_set=valid_data,
                                                      metric=metric,
                                                      lr=learning_rate,
                                                      nl=num_leaves,
                                                      md=max_depth,
                                                      bf=bagging_fraction,
                                                      ff=feature_fraction,
                                                      mcw=min_child_weight,
                                                      mdil=min_data_in_leaf,
                                                      l1=lambda_l1,
                                                      l2=lambda_l2)
        oofs[valid_idx] = model.predict(X_train.iloc[valid_idx,:], num_iteration=model.best_iteration)
        models.append(model)
        learning_curves.append(learning_curve)
        valid_score.append(best_score[f'valid_{metric}'])
        # best_scores.append(best_score)
        gc.collect()
    
    valid_score_avg = np.mean(valid_score)
    valid_score_std = np.std(valid_score)
    
    if bayes_opt:
        return -valid_avg_score
    else:
        return valid_score_avg, valid_score_std, models, oofs #best_scores, learning_curves
    
def lgb_reg_bayes_opt(init_points=20, n_iteration=80):
    bounds = {'learning_rate': (0.001, 0.3),
              'num_leaves': (20, 500), 
              'bagging_fraction' : (0.1, 1),
              'feature_fraction' : (0.1, 1),
              'min_child_weight': (0.001, 0.99),   
              'min_data_in_leaf': (3, 200),
              'max_depth': (-1, 100),
              'lambda_l1': (0.1, 300), 
              'lambda_l2': (0.1, 300)}
    
    optimizer = BayesianOptimization(f=lgb_kfold, pbounds=bounds, random_state=8982)
    optimizer.maximize(init_points=init_points, n_iter=n_iteration)
    
    print('Best score:', -optimizer.max['target'])
    print('Best set of parameters:')
    print(optimizer.max['params'])
    param = optimizer.max['params']; cv = -optimizer.max['target']
    return param, cv

In [ ]:
def pred(X_test, models):
    y_test_pred_total = np.zeros(X_test.shape[0])
    for i, model in enumerate(models):
        print(f'Predicting with {i}-th model')
        y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)
        y_test_pred_total += y_pred_test
    y_test_pred_total /= len(models)
    return y_test_pred_total

### CatBoost

In [23]:
import catboost as cb

In [24]:
def cb_kfold_clf(
    params,
    X_train,
    y_train,
    X_test,
    cv,
    features,
    metrics,
    categorical=[],
):
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_test))
    feature_importance_df = pd.DataFrame()
    
    n_splits = len(cv)
    printl(f"k={n_splits} folds catboost running...")
    printl(f"train data/feature shape: {X_train[features].shape}")
    
    for fold, (dev_idx, val_idx) in enumerate(cv):
        dev_data = cb.Pool(X_train.loc[dev_idx, features],
                           label=y_train[dev_idx],
                           cat_features=categorical)
        val_data = cb.Pool(X_train.loc[val_idx, features],
                           label=y_train[val_idx],
                           cat_features=categorical)
        
        clf = cb.CatBoostClassifier(**params)
        clf.fit(dev_data, eval_set=val_data)
        time.sleep(1)
        
        oof[val_idx] = clf.predict_proba(X_train.loc[val_idx, features])[:,1]
        predictions += clf.predict_proba(X_test[features])[:,1] / n_splits
        
        msg = f'fold: {fold}'
        for name, func in metrics.items():
            score = func(y_train[val_idx], oof[val_idx])
            msg += f' - {name}: {score:.5f}'
        printl(msg)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = features
        fold_importance_df['gain'] = clf.get_feature_importance()
        fold_importance_df['fold'] = fold
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    msg = f'CV score'
    for name, func in metrics.items():
        score = func(y_train, oof)
        msg += f' - {name}: {score:.5f}'
    printl(msg)

    return oof, predictions, feature_importance_df


def cb_kfold_reg(
    params,
    X_train,
    y_train,
    X_test,
    cv,
    features,
    metrics,
    categorical=[],
):
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_test))
    feature_importance_df = pd.DataFrame()
    
    n_splits = len(cv)
    printl(f"k={n_splits} folds catboost running...")
    printl(f"train data/feature shape: {X_train[features].shape}")
    
    for fold, (dev_idx, val_idx) in enumerate(cv):
        dev_data = cb.Pool(X_train.loc[dev_idx, features],
                           label=y_train[dev_idx],
                           cat_features=categorical)
        val_data = cb.Pool(X_train.loc[val_idx, features],
                           label=y_train[val_idx],
                           cat_features=categorical)
        
        clf = cb.CatBoostRegressor(**params)
        clf.fit(dev_data, eval_set=val_data)
        time.sleep(1)
        
        oof[val_idx] = clf.predict(X_train.loc[val_idx, features])
        predictions += clf.predict(X_test[features]) / n_splits
        
        msg = f'fold: {fold}'
        for name, func in metrics.items():
            score = func(y_train[val_idx], oof[val_idx])
            msg += f' - {name}: {score:.5f}'
        printl(msg)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = features
        fold_importance_df['gain'] = clf.get_feature_importance()
        fold_importance_df['fold'] = fold
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    msg = f'CV score'
    for name, func in metrics.items():
        score = func(y_train, oof)
        msg += f' - {name}: {score:.5f}'
    printl(msg)

    return oof, predictions, feature_importance_df

### Neural Net

In [25]:
import tensorflow as tf
import tensorflow.keras as keras
# import torch

In [26]:
def build_neuralnet(
    recipe,
    loss='mse',
    optimizer='adam',
    lr=1e-3,
    monitor='val_loss',
    es_patience=-1,
    restore_best_weights=True,
    lr_scheduler='none',
    lr_factor=0.1,
    lr_patience=5,
    seed=42,
    **_,
):
    tf.random.set_seed(seed)
    model = keras.models.model_from_json(recipe)
    
    if loss == 'mse':
        loss = keras.losses.mean_squared_error
    elif loss == 'bce':
        loss = keras.losses.binary_crossentropy
    else:
        raise NotImplementedError
    
    if optimizer == 'adam':
        optimizer = keras.optimizers.Adam(lr)
    else:
        raise NotImplementedError
    
    model.compile(optimizer=optimizer, loss=loss)
    
    callbacks = []
    
    if es_patience >= 0:
        es = keras.callbacks.EarlyStopping(monitor=monitor,
                                           patience=es_patience,
                                           restore_best_weights=restore_best_weights,
                                           verbose=1)
        callbacks.append(es)
    
    if lr_scheduler == 'none':
        pass
    elif lr_scheduler == 'reduce_on_plateau':
        lr_sche = keras.callbacks.ReduceLROnPlateau(monitor=monitor,
                                                    factor=lr_factor,
                                                    patience=lr_patience,
                                                    verbose=1)
        callbacks.append(lr_sche)
    else:
        raise NotImplementedError
    
    return model, callbacks


def train_neuralnet(
    params,
    X_train,
    y_train,
    validation_data=None,
):
    model, callbacks = build_neuralnet(**params)
    model.fit(X_train,
              y_train,
              validation_data=validation_data,
              batch_size=params['batch_size'],
              epochs=params['epochs'],
              callbacks=callbacks)
    return model


def run_kfold_neuralnet(
    params,
    X_train,
    y_train,
    X_test,
    cv,
    features,
    metrics,
):
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_test))
    
    n_splits = len(cv)
    printl(f"k={n_splits} folds neuralnet running...")
    printl(f"train data/feature shape: {X_train[features].shape}")
    
    for fold, (dev_idx, val_idx) in enumerate(cv):
        validation_data = [X_train.loc[val_idx, features], y_train[val_idx]]
        model = train_neuralnet(params,
                                X_train.loc[dev_idx, features],
                                y_train[dev_idx],
                                validation_data=validation_data)
        
        oof[val_idx] = model.predict(X_train.loc[val_idx, features].values)[:,0]
        predictions += model.predict(X_test[features].values)[:,0] / n_splits
        
        msg = f'fold: {fold}'
        for name, func in metrics.items():
            score = func(y_train[val_idx], oof[val_idx])
            msg += f' - {name}: {score:.5f}'
        printl(msg)
    
    msg = f'CV score'
    for name, func in metrics.items():
        score = func(y_train, oof)
        msg += f' - {name}: {score:.5f}'
    printl(msg)

    return oof, predictions

## Feature Selection

### Plot Feature Importance

In [27]:
def plot_feature_importance(
    feature_importance_df,
    feature_name='feature',
    importance_name=['split', 'gain'],
    top_k=50,
    fig_width=16,
    fig_height=8,
    fontsize=14,
):
    if isinstance(importance_name, str):
        importance_name = [importance_name]
    
    num_importance = len(importance_name)
    plt.figure(figsize=(fig_width, fig_height*num_importance))
    gs = gridspec.GridSpec(1, num_importance)
    
    def _fetch_best_features(df, fimp='gain'):
        cols = (df[[feature_name, fimp]]
                .groupby(feature_name)
                .mean()
                .sort_values(by=fimp, ascending=False)
                .index
                .values[:top_k])
        return cols, df.loc[df[feature_name].isin(cols)]
    
    for i, fimp in enumerate(importance_name):
        cols, best_features = _fetch_best_features(feature_importance_df, fimp)
        ax = plt.subplot(gs[0, i])
        sns.barplot(x=fimp, y=feature_name, data=best_features, order=cols, ax=ax)
        title = f'Features {fimp} importance (averaged/folds)'
        plt.title(title, fontweight='bold', fontsize=fontsize)
    
    plt.tight_layout()

### Iterative Feature Elimination by LightGBM

In [28]:
class Iterative_CV:
    def __init__(self, X_train_full, y_train, eval_cols, metric):
        self.X_train_full = X_train_full
        self.y_train = y_train
        self.eval_cols = eval_cols
        self.metric = metric
        
    def iter_cv_elim():
        excl_improve = []; excl_worse = []
        if self.metric == 'rmse':
            init_valid_avg_score = -1 * lgb_kfold(self.X_train_full, self.y_train, bayes_opt=True)
            print(f'[Iter_Feature_Elim] Current best score is {init_valid_avg_score}')
            for cols in tqdm(self.eval_cols):
                temp_cols = list(set(self.X_train_full.columns) - {col})
                X_train = self.X_train_full[temp_cols]
                new_valid_avg_score = -1 * lgb_kfold(X_train, self.y_train, bayes_opt=True)
                degree = new_valid_avg_score - init_valid_avg_score
                if degree < 0:
                    pct = 100 * (-1 * degree / init_valid_avg_score)
                    excl_improve.append([col, pct])
                    print(f"[Iter_Feature_Elim] '{col}', exclusion improved (lowered) avg CV by {pct}pct.")
                else:
                    pct = 100 * (degree / init_valid_avg_score)
                    excl_worse.append([col, pct])
                    print(f"[Iter_Feature_Elim] '{col}', exclusion worsened (raised) avg CV by {pct}pct.")
        elif self.metric == 'auc':
            init_valid_avg_score = lgb_skfold(self.X_train_full, self.y_train, bayes_opt=True)
            print(f'[Iter_Feature_Elim] Current best score is {init_valid_avg_score}')
            for col in tqdm(self.eval_cols):
                temp_cols = list(set(self.X_train_full.columns) - {col})
                X_train = self.X_train_full[temp_cols]
                new_valid_avg_score = lgb_skfold(X_train, self.y_train, bayes_opt=True)
                degree = new_valid_avg_score - init_valid_avg_score
                if degree > 0:
                    pct = 100 * (degree / init_valid_avg_score)
                    excl_improve.append([col, pct])
                    print(f"[Iter_Feature_Elim] '{col}', exclusion improved (raised) avg CV by {pct}pct.")
                else:
                    pct = 100 * (-1 * degree / init_valid_avg_score)
                    excl_worse.append([col, pct])
                    print(f"[Iter_Feature_Elim] '{col}', exclusion worsened (lowered) avg CV by {pct}pct.")

        excl_improve.sort(key=lambda lst: lst[1])
        excl_worse.sort(key=lambda lst: lst[1])
        del init_valid_avg_score, cols, temp_cols, X_train, new_valid_avg_score, degree, pct
        gc.collect()
        return excl_improve, excl_worse
    
    def iter_cv_rank():
        impt = []
        if self.metric == 'rmse':
            for col in tqdm(self.eval_cols):
                X_train = self.X_train_full[col]
                assert X_train.shape[1] == 1
                print(f"[Iter_Feature_Rank] '{col}', evaluation ongoing.")
                valid_avg_score = -1 * lgb_kfold(X_train, self.y_train, bayes_opt=True)
                impt.append([col, valid_avg_score])
        elif self.metric == 'auc':
            for col in tqdm(self.eval_cols):
                X_train = self.X_train_full[col]
                assert X_train.shape[1] == 1
                print(f"[Iter_Feature_Rank] '{col}', evaluation ongoing.")
                valid_avg_score = lgb_skfold(X_train, self.y_train, bayes_opt=True)
                impt.append([col, valid_avg_score])
        impt.sort(key=lambda lst: lst[1])
        del col, X_train, valid_avg_score
        gc.collect()
        return impt

### Null Importance Selection

In [29]:
def _get_lgb_fimp(
    params,
    X_train,
    y_train,
    features,
    shuffle,
    seed=42,
    categorical=[]
):
    # Shuffle target if required
    y = y_train.copy()
    if shuffle:
        random.seed(seed)
        np.random.seed(seed)
        y = y_train.copy().sample(frac=1.0)
    
    arg_categorical = categorical if len(categorical) > 0 else 'auto'
    dtrain = lgb.Dataset(X_train[features],
                         label=y.values,
                         categorical_feature=arg_categorical)
    
    # Fit the model
    clf = lgb.train(params, dtrain)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df['feature'] = features
    imp_df['split'] = clf.feature_importance(importance_type='split')
    imp_df['gain'] = clf.feature_importance(importance_type='gain')
    
    return imp_df


def null_importance_selection(
    params,
    X_train,
    y_train,
    features,
    seed=42,
    categorical=[],
    num_actual_run=1,
    num_null_run=40,
    eps=1e-10,
    valid_percentile=75,
):
    actual_imp_df = pd.DataFrame()
    
    np.random.seed(seed)
    for i in tqdm(range(num_actual_run)):
        seed = np.random.randint(1000)
        imp_df = _get_lgb_fimp(params,
                               X_train,
                               y_train,
                               features,
                               shuffle=False,
                               seed=seed,
                               categorical=categorical)
        imp_df['run'] = i
        actual_imp_df = pd.concat([actual_imp_df, imp_df], axis=0)
    
    null_imp_df = pd.DataFrame()
    
    np.random.seed(seed)
    for i in tqdm(range(num_null_run)):
        seed = np.random.randint(1000)
        imp_df = _get_lgb_fimp(params,
                               X_train,
                               y_train,
                               features,
                               shuffle=True,
                               seed=seed,
                               categorical=categorical)
        imp_df['run'] = i
        null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    
    feature_scores = []
    
    for _f in actual_imp_df['feature'].unique():
        # importance gain of gain
        act_fimp_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'split'].mean()
        null_fimp_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'split'].values
        split_score = np.log(eps + act_fimp_split / (1 + np.percentile(null_fimp_split, valid_percentile)))
        
        # importance gain of gain
        act_fimp_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'gain'].mean()
        null_fimp_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'gain'].values
        gain_score = np.log(eps + act_fimp_gain / (1 + np.percentile(null_fimp_gain, valid_percentile)))

        feature_scores.append((_f, split_score, gain_score))
    
    scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])
    return scores_df

### Consider Multicollinearity

In [30]:
def extract_high_corr_columns(df, threshold=0.99, verbose=True):
    df_corr = abs(df.corr())
    delete_columns = []
    
    # diagonal values filled by zero
    for i in range(0, len(df_corr.columns)):
        df_corr.iloc[i, i] = 0
    
    # loop as removing high-correlated columns in df_corr
    while True:
        df_max_column_value = df_corr.max()
        max_corr = df_max_column_value.max()
        query_column = df_max_column_value.idxmax()
        target_column = df_corr[query_column].idxmax()
        
        if max_corr < threshold:
            break
        else:
            # drop feature which is highly correlated with others 
            if sum(df_corr[query_column]) <= sum(df_corr[target_column]):
                delete_column = target_column
                saved_column = query_column
            else:
                delete_column = query_column
                saved_column = target_column
            
            df_corr.drop([delete_column], axis=0, inplace=True)
            df_corr.drop([delete_column], axis=1, inplace=True)
            delete_columns.append(delete_column)
            
            if verbose:
                printl('{}: Drop: {} <- Query: {}, Corr: {:.5f}'.format(
                    len(delete_columns), delete_column, saved_column, max_corr
                ))

    return delete_columns